<a href="https://colab.research.google.com/github/jaytrey777/Classification-Trees/blob/main/Classification_Trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tasks

Uue the following classifiers to obtain the highest accuracy possible on the test set:

1. Decision Tree Classifier

2. Bagging Classifier

3. Random Forest Classifier

Link to original dataset:
https://drive.google.com/file/d/1Zms1RfgkWrTp7S6_BFpeELYyvX1s2FSN/view

# Imports

In [ ]:

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier


# Data Loading & Cleaing

# Preprocessing

# Decision Tree Classifier

#Bagging Classifier

# Random Forest Classifier

# Analysis

# Optimization

# Final Analysis